In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class YOLOv8Dataset(Dataset):
    def __init__(self, root_dir, mode, class_mapping, img_size=640, transforms=None):
        self.root_dir = root_dir
        self.mode = mode
        self.img_dir = os.path.join(root_dir, mode, 'images')
        self.label_dir = os.path.join(root_dir, mode, 'labels')
        self.class_mapping = class_mapping
        self.img_size = img_size
        self.transforms = transforms
        self.imgs = [f for f in os.listdir(self.img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        label_path = os.path.join(self.label_dir, self.imgs[idx].rsplit('.', 1)[0] + '.txt')
        
        img = Image.open(img_path).convert("RGB")
        
        # Read labels
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                labels = np.array([x.split() for x in f.read().strip().splitlines()], dtype=np.float32)
        else:
            labels = np.array([])

        # Normalization for YOLO format
        if len(labels):
            labels[:, 1:] = np.clip(labels[:, 1:], 0, 1)

        # Apply transforms
        if self.transforms:
            img = self.transforms(img)

        # Convert to tensor
        nL = len(labels)
        labels_out = torch.zeros((nL, 6))
        if nL:
            labels_out[:, 1:] = torch.from_numpy(labels)

        return img, labels_out

    @staticmethod
    def collate_fn(batch):
        img, label = zip(*batch)
        for i, l in enumerate(label):
            l[:, 0] = i  # add target image index for build_targets()
        return torch.stack(img, 0), torch.cat(label, 0)

def create_dataloader(root_dir, mode, class_mapping, batch_size, img_size=640, num_workers=0):
    dataset = YOLOv8Dataset(
        root_dir=root_dir,
        mode=mode,
        class_mapping=class_mapping,
        img_size=img_size,
        transforms=transforms.Compose([
            transforms.ToTensor(),
        ])
    )
    
    return DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=(mode == 'train'),  # shuffle only if it's training data
        num_workers=num_workers, 
        collate_fn=YOLOv8Dataset.collate_fn
    )

def test_dataloader_func(dataloader):
    for imgs, labels in dataloader:
        for i in range(len(imgs)):
            # Select the first image in the batch
            img = imgs[i]
            img = img.permute(1, 2, 0)
            img_np = img.numpy()

            # Display image
            fig, ax = plt.subplots(1)
            ax.imshow(img_np)
            ax.axis('off')

            # Draw bounding boxes
            for label in labels[labels[:, 0] == i]:
                _, cls, x, y, w, h = label.tolist()
                x1, y1 = x - w/2, y - h/2
                x2, y2 = x + w/2, y + h/2
                rect = patches.Rectangle((x1 * img.shape[1], y1 * img.shape[0]), 
                                         w * img.shape[1], h * img.shape[0], 
                                         linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)
                ax.text(x1 * img.shape[1], y1 * img.shape[0], str(int(cls)), 
                        fontsize=8, verticalalignment='top', 
                        bbox=dict(facecolor='red', alpha=0.5))

            plt.show()

            print("Image", i)
            print("Image shape:", imgs[i].shape)
            print("Number of boxes:", len(labels[labels[:, 0] == i]))
            print("Labels:", labels[labels[:, 0] == i])
        break  # Test only the first batch

# Example class mapping dictionary
class_mapping = {
    'apple': 0,
}

# Set your root directory
root_dir = './'

# Create dataloaders for each mode
train_dataloader = create_dataloader(root_dir, 'train', class_mapping, batch_size=10)
valid_dataloader = create_dataloader(root_dir, 'valid', class_mapping, batch_size=10)
test_dataloader = create_dataloader(root_dir, 'test', class_mapping, batch_size=10)

# Test the dataloaders
# print("Testing train dataloader:")
# test_dataloader_func(train_dataloader)

# print("Testing valid dataloader:")
# test_dataloader_func(valid_dataloader)

# print("Testing test dataloader:")
# test_dataloader_func(test_dataloader)

In [3]:
from ultralytics import YOLO
import yaml
import torch

# 데이터셋 경로와 클래스 이름 설정
data_dir = './'
class_names = ['green apple', 'red apple']  # 클래스 이름 리스트

# 모델 생성
model = YOLO('yolov8n.yaml')  # YOLOv8n 모델 구조 로드

# 모델 학습
results = model.train(
    data='data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,
    name='yolov8n_custom'
)

# 학습된 모델 저장 (수정된 부분)
if hasattr(model, 'model') and hasattr(model.model, 'state_dict'):
    model.save('yolov8n_custom.pt')
else:
    print("모델 상태를 저장할 수 없습니다. 학습이 제대로 완료되었는지 확인하세요.")

# 테스트 데이터셋에 대한 평가
results = model.val()

Ultralytics YOLOv8.2.53  Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning C:\Users\admin\Desktop\0711\train\labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
val: Scanning C:\Users\admin\Desktop\0711\valid\labels.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<?, ?it/s]


Plotting labels to C:\Users\admin\Desktop\runs\detect\yolov8n_custom7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\admin\Desktop\runs\detect\yolov8n_custom7
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/40      2.46G      3.419      3.911      4.201         43        640: 100%|██████████| 2/2 [00:07<00:00,  3.61s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.78s/it]

                   all          2         28          0          0          0          0


...


40 epochs completed in 0.014 hours.
Optimizer stripped from C:\Users\admin\Desktop\runs\detect\yolov8n_custom7\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\admin\Desktop\runs\detect\yolov8n_custom7\weights\best.pt, 6.2MB

Validating C:\Users\admin\Desktop\runs\detect\yolov8n_custom7\weights\best.pt...
Ultralytics YOLOv8.2.53  Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 56.86it/s]

                   all          2         28    0.00103     0.0192   0.000537   5.37e-05
           green apple          1          2          0          0          0          0
             red apple          2         26    0.00206     0.0385    0.00107   0.000107


Speed: 0.0ms preprocess, 3.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to C:\Users\admin\Desktop\runs\detect\yolov8n_custom7


TypeError: 'NoneType' object is not a mapping

In [13]:
from ultralytics import YOLO

# 사전 학습된 모델 로드
model = YOLO('yolov8n.pt')  # 또는 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt'

# 이미지에 대한 예측
results = model('apple.jpg')

# 결과 시각화
results[0].show()  # 또는 results[0].plot()

KeyError: 'model'

---

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(data="./data.yaml", epochs=100)

In [3]:
from ultralytics import YOLO

# 수정된 YOLOv8 모델 초기화
model = YOLO('yolov8n.yaml')

# 모델 학습
model.train(data="./data.yaml", epochs=100)


Ultralytics YOLOv8.2.53  Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=./data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning C:\Users\admin\Desktop\0711\train\labels... 375 images, 0 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<00:00, 2461.81it/s]

train: New cache created: C:\Users\admin\Desktop\0711\train\labels.cache



val: Scanning C:\Users\admin\Desktop\0711\valid\labels... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<00:00, 1666.31it/s]

val: New cache created: C:\Users\admin\Desktop\0711\valid\labels.cache


...

                   all         35         69      0.862          1      0.962      0.828



100 epochs completed in 0.081 hours.
Optimizer stripped from C:\Users\admin\Desktop\runs\detect\train5\weights\last.pt, 5.4MB
Optimizer stripped from C:\Users\admin\Desktop\runs\detect\train5\weights\best.pt, 5.4MB

Validating C:\Users\admin\Desktop\runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.2.53  Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLOv8n summary (fused): 90 layers, 2639185 parameters, 0 gradients, 4.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.39it/s]


                   all         35         69      0.907      0.984      0.971      0.836
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to C:\Users\admin\Desktop\runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000252894DEFE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [2]:
import torch
model = torch.load("./best.pt")
print(model)

{'date': '2024-07-12T11:45:40.921618', 'version': '8.2.53', 'license': 'AGPL-3.0 (https://ultralytics.com/license)', 'docs': 'https://docs.ultralytics.com', 'epoch': -1, 'best_fitness': None, 'model': DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Co

In [8]:
from ultralytics import YOLO

# Load a model
model = YOLO("best.pt")  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(["apple5.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


0: 480x640 10 apples, 14.0ms
Speed: 4.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


In [4]:
%cd C:/Users/admin/Desktop/0711/yolo_weights/
!echo %cd%

C:\Users\admin\Desktop\0711\yolo_weights
C:\Users\admin\Desktop\0711\yolo_weights


In [1]:
import torch
from ultralytics import YOLO

# ------------ 모델 경로 및 파일 수정 시간은 수정하기 --------------
model_path = '../best.pt' 
model_generated_time = "  24-07-12, 12:30"
# --------------------------------------------------------------

model = YOLO(model_path)

# 모델의 가중치 확인
model_weights = model.model.state_dict()
torch.set_printoptions(threshold=torch.inf)
idx = 0
while idx < len(model_weights):
    name = list(model_weights.keys())[idx]
    param = model_weights[name]
    if "model." in name:
        output_file = f"{name}.txt" 
        with open(output_file, "w") as f:
            f.write("Weights Information:\n")
            f.write("# weights ver.2\n\n")
            f.write(f"Model Path: {model_path}\n")
            f.write(f"Model Generated Time: {model_generated_time}\n\n")
            f.write(f"Layer: {name} | Size: {param.size()}\n")
            f.write(str(param) + "\n")  
    idx += 1

KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomBatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True):
        super(CustomBatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        
        # Parameters for affine transformation
        if self.affine:
            self.gamma = nn.Parameter(torch.ones(num_features))
            self.beta = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter('gamma', None)
            self.register_parameter('beta', None)
        
        # Buffers for running statistics
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        
    def forward(self, x):
        if self.training:
            # Calculate mean and variance for the current batch
            mean = x.mean([0, 2, 3])  # Mean over N, H, W
            var = x.var([0, 2, 3], unbiased=False)  # Variance over N, H, W
            
            # Update running statistics
            self.running_mean = self.momentum * mean + (1 - self.momentum) * self.running_mean
            self.running_var = self.momentum * var + (1 - self.momentum) * self.running_var
            
            # Normalize the input
            x_hat = (x - mean[None, :, None, None]) / torch.sqrt(var[None, :, None, None] + self.eps)
        else:
            # Use running statistics during inference
            mean = self.running_mean
            var = self.running_var
            x_hat = (x - mean[None, :, None, None]) / torch.sqrt(var[None, :, None, None] + self.eps)
        
        # Apply affine transformation
        if self.affine:
            x_hat = self.gamma[None, :, None, None] * x_hat + self.beta[None, :, None, None]
        
        return x_hat

# Example usage
if __name__ == "__main__":
    # Create an instance of the CustomBatchNorm2d
    batch_norm = CustomBatchNorm2d(num_features=16, eps=1e-5, momentum=0.1, affine=True)
    
    # Create a sample input tensor (batch_size, num_channels, height, width)
    input_tensor = torch.randn(8, 16, 2, 2)  # e.g., (batch_size=8, num_channels=16, height=32, width=32)
    
    # Forward pass through the custom batch normalization layer
    output_tensor = batch_norm(input_tensor)
    
    print(input_tensor[0])
    print(output_tensor[0])  

tensor([[[ 0.9654, -0.4140],
         [-0.2186, -0.9651]],

        [[ 1.5843,  0.3348],
         [ 0.4944, -0.1666]],

        [[-1.2554,  0.9271],
         [ 0.1838, -0.8233]],

        [[-0.3026, -0.2514],
         [ 0.8208,  0.7560]],

        [[ 0.5401, -0.1063],
         [ 0.7480,  2.6830]],

        [[ 0.3632, -0.0683],
         [ 1.2184, -0.3892]],

        [[ 0.7510,  0.4972],
         [-0.3731, -1.5629]],

        [[-0.4807,  0.3936],
         [-1.5347,  0.0213]],

        [[ 0.0606, -2.5097],
         [-0.8907,  0.5267]],

        [[ 0.1400,  1.7042],
         [ 0.0561,  1.0140]],

        [[ 1.1550,  1.5153],
         [ 0.1052,  0.5345]],

        [[ 0.0428,  0.3632],
         [-0.4449, -1.7511]],

        [[-0.2093, -0.9839],
         [ 0.3099,  1.3476]],

        [[ 0.6045, -0.6123],
         [ 1.7775,  0.4024]],

        [[ 2.1645, -1.5787],
         [ 1.3314, -0.4347]],

        [[-0.7302,  0.8248],
         [ 0.3284,  1.0966]]])
tensor([[[ 0.8716, -0.2711],
         [-